In [1]:
import datetime
from functools import reduce
import pandas as pd
from pytrends.request import TrendReq
from pytz import timezone
import numpy as np
import os
import re

# Google trends 

In [2]:
test_trends1 = ["anxieté", "depression", "suicide", "bipolaire", "angoisse", "sertraline" ]

test_trends2 = ["sensation", "futur", "optimisme", "depression", "ambiguite"]

In [3]:
pytrends = TrendReq(hl="en-US", tz=360)

In [4]:
#can drop this function in the future , not very useful for now 
def set_date_range(start, end, interval):
    difference = (end  - start ) / interval
    for i in range(interval):
        yield (start + difference *i).strftime("%Y%m%d")
    yield end.strftime("%Y%m%d")

In [5]:
country_dataframes = []
for country_file in os.listdir("country_covid"):
    country_dataframes.append(pd.read_csv(f'country_covid/{country_file}'))

covid_france = country_dataframes[0]
covid_germany = country_dataframes[1]
covid_uk =  country_dataframes[2]

In [6]:
#Get time frame of covid dataframes 
daily_timeframe = pd.date_range(start=country_dataframes[0]["date"].min(), 
                                end=country_dataframes[0]["date"].max(),
                                freq="D",
                                name="daily_timeframe")

In [7]:
#need to create 3 time frames:

# "timeframe" = the entire time frame of country_dataframe I.E the time range of the data we got
#                                                                               for each country

timeframe = str(daily_timeframe.min().strftime("%Y-%m-%d"))
timeframe = timeframe + " " + str(daily_timeframe.max().strftime("%Y-%m-%d"))


# we must split the timeframe in two. Other wise Google trends does not provide daily data
# for such a long time

mid_point = list(set_date_range(daily_timeframe.min(),
                                daily_timeframe.max(), interval=2))[1]

time_half_1 = str(daily_timeframe.min().strftime("%Y-%m-%d")) 
time_half_1 = time_half_1 + " " + str(pd.to_datetime(mid_point).strftime("%Y-%m-%d"))

time_half_2 = str((pd.to_datetime(mid_point)+datetime.timedelta(days=1)).strftime("%Y-%m-%d")) 
time_half_2 = time_half_2 + " " + str(daily_timeframe.max().strftime("%Y-%m-%d"))

In [8]:
def fetch_word(timeframe: list):
    kw_word = [input("Enter word to add to df: ")]
    pytrends.build_payload(kw_list=kw_word,
                                   cat=0,
                                   timeframe=timeframe,
                                   geo="FR",
                                   gprop="")
    data = pytrends.interest_over_time()
    data.reset_index(inplace=True)
    data.drop("isPartial", axis=1, inplace=True)
    return data

In [9]:
#by default : google trends aggregates time period to weeks if time period > 7 months & < 2 years
#this is weekly data for a time_frame this size 

def fetch_data(timeframe: str, words: list): 
    trends = []
    for word in words:
        pytrends.build_payload(kw_list = [word],
                               cat=0,
                               timeframe=timeframe,
                               geo="FR",
                               gprop="")
        word_trend = pytrends.interest_over_time()
        word_trend.reset_index(inplace=True)
        word_trend.drop("isPartial", axis=1, inplace=True)
        
        trends.append(word_trend)
        
    trends_merged = reduce(lambda trend_left, trend_right: pd.merge(trend_left,
                                                                    trend_right,
                                                                    on="date",
                                                                    how="inner"), trends)
    return trends_merged

In [10]:
def fetch_daydata(words :list):
    
    day_data = fetch_data(time_half_1, words).append(fetch_data(time_half_2, words))
    day_data = day_data.reset_index().drop("index", axis=1)
    
    return day_data

In [15]:
def aggregate_data(words : list):
    
    clean_words = []
    clean_word_trends = []
    agg_trends = []

    dirty_trends = pd.merge(fetch_daydata(words), fetch_data(timeframe, words),
                             on="date",
                             how="outer",
                             suffixes=["_day", "_week"])
    dirty_trends.fillna(method="ffill", inplace=True)
    
    base_timeframe = dirty_trends[["date"]]
    base_timeframe.reset_index(inplace=True)
    
    for word in words:
        clean_words.append(word.split("_")[0])
    
    for clean_word in clean_words:
        
        #problem here with regex below. It isn't perfect a perfect match regex 
        #need to change in the futur
        #if user types similar words in google trend research bar , for example : 
        # "depression" AND "deprime" then filter takes 4 columns instead of 2
        
        clean_trends = dirty_trends.filter(regex=clean_word, axis=1)
        clean_word_trends.append(clean_trends)

    for trend in clean_word_trends:
        daily_value = trend[trend.columns[0]]
        daily_value.replace([0], 1, inplace = True)
        
        weekly_value = trend[trend.columns[1]]
        weekly_value.replace([0] , 1, inplace=True)
        
        trend.dropna(inplace=True)
        trend_name = trend.columns[0].split("_")[0]
        
        trend[f"agg_{trend_name}"] = daily_value * weekly_value / 100

    for trend in clean_word_trends:
        agg_columns = trend.filter(regex="agg_", axis=1)
        agg_trends.append(agg_columns)

    agg_dataframe = reduce(lambda trend_left, trend_right: trend_left.join(trend_right), 
                                        agg_trends)
    agg_dataframe.reset_index(inplace=True)
    agg_dataframe = base_timeframe.merge(agg_dataframe, on = "index", how = "right")
    
    return agg_dataframe

In [16]:
aggregate_data(test_trends1)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\matth\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\matth\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,date,agg_anxieté,agg_depression,agg_suicide,agg_bipolaire,agg_angoisse,agg_sertraline
0,4,2020-03-22,0.01,34.84,10.92,8.17,75.05,10.36
1,5,2020-03-23,0.01,31.49,6.72,6.88,88.35,12.58
2,6,2020-03-24,0.01,32.16,5.04,16.34,42.75,4.44
3,7,2020-03-25,0.01,34.17,9.24,11.18,73.15,0.37
4,8,2020-03-26,0.01,37.52,9.66,11.18,79.80,4.44
...,...,...,...,...,...,...,...,...
385,389,2021-04-11,0.01,51.60,16.43,14.11,30.00,14.25
386,390,2021-04-12,0.01,30.10,15.37,33.20,33.00,19.95
387,391,2021-04-13,0.01,36.98,7.95,29.88,33.00,6.84
388,392,2021-04-14,0.01,43.86,11.13,50.63,21.50,27.36
